In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from netam import noof, shmoof
from epam.torch_common import pick_device

In [2]:
full_shmoof_df = pd.read_csv("/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv", index_col=0).reset_index(drop=True)

full_shmoof_df = full_shmoof_df.sample(5000)

train_df = full_shmoof_df.sample(frac=0.8)
val_df = full_shmoof_df.drop(train_df.index)

kmer_length = 3
max_length = 500

train_dataset = shmoof.SHMoofDataset(train_df, kmer_length=kmer_length, max_length=max_length)
val_dataset = shmoof.SHMoofDataset(val_df, kmer_length=kmer_length, max_length=max_length)

In [3]:
model = noof.NoofModel(train_dataset, embedding_dim=2, nhead=2, dim_feedforward=512, layer_count=3, dropout=0.5)

device = pick_device()
train_dataset.to(device)
val_dataset.to(device)
model.to(device)

burrito = noof.NoofBurrito(train_dataset, val_dataset, model, l2_regularization_coeff=1e-6)
print("starting training...")
losses = burrito.train(epochs=3)

Using Metal Performance Shaders
starting training...
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/matsen/mambaforge/envs/epam/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/16/pkph6n2962g3lf2wpwwt0wwr0000gn/T/ipykernel_34191/1382760333.py", line 10, in <module>
    losses = burrito.train(epochs=3)
  File "/Users/matsen/re/netam/netam/shmoof.py", line 219, in train
    }
  File "/Users/matsen/re/netam/netam/shmoof.py", line 254, in _calculate_loss
  File "/Users/matsen/mambaforge/envs/epam/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/Users/matsen/mambaforge/envs/epam/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/Users/matsen/mambaforge/envs/epam/lib/python3.9/site-packages/torch/nn/modules/loss.py", line 618, in forward
    return F.binary_c